In [ ]:
'''
Created on Feb 21, 2019

@author: Iswarya Vardhini
'''
import pandas as pd
from distributed import Client, progress
from hdfs import InsecureClient
import os 
import projpk.masdriv as masdriv
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import ttest_ind
import scipy.stats as stats
from scipy import stats


def seatprocess():
    seatdf = masdriv.df
    print ("Data Cleaning process starts in Seatbelt program")
    eduwn = seatdf[seatdf._EDUCAG.notnull()]
    agewn = eduwn[eduwn._AGEG5YR.notnull()]
    seatwn = agewn[agewn.SEATBELT.notnull()]
    seatwn=seatwn[seatwn._AGEG5YR != 14]
    seatwn=seatwn[seatwn._STATE == 47]
    seatwn=seatwn[(seatwn._EDUCAG != 5) & (seatwn._EDUCAG != 6)& (seatwn._EDUCAG != 9)]
    #seatwn=seatwn[seatwn._INCOMG != 9]
    #seatwn = seatwn[(seatwn._AGE65YR != 3) & (seatwn._AGE65YR != 4)& (seatwn._AGE65YR != 5)& (seatwn._AGE65YR != 6)& (seatwn._AGE65YR != 7)& (seatwn._AGE65YR != 8)& (seatwn._AGE65YR != 9)& (seatwn._AGE65YR != 10)& (seatwn._AGE65YR != 11)& (seatwn._AGE65YR != 12)& (seatwn._AGE65YR != 13)]
    seatwn = seatwn[(seatwn._RFSEAT3 != 3) & (seatwn._RFSEAT3 != 4) & (seatwn._RFSEAT3 != 9)]
    seatwn = seatwn[(seatwn.SEATBELT != 7) & (seatwn.SEATBELT != 8) & (seatwn.SEATBELT != 9)]
    
        
    '''print (seatwn.groupby(['_AGE65YR']).count())
    print (seatwn.groupby(['_RFSEAT3']).count())'''
    
    
    '''seatwn.set_index('_AGEG5YR', inplace = True)
    all3swioutnulldf = seatwn
    print (all3swioutnulldf.groupby(['_AGEG5YR']).count())
    all3swioutnulldf.drop(14, inplace = True)
    print (all3swioutnulldf.groupby(['_AGEG5YR']).count())
    all3swioutnulldf.reset_index(level = ['_AGEG5YR'])
    print (all3swioutnulldf.groupby(['_INCOMG']).count())
    all3swioutnulldf.set_index('_INCOMG', inplace = True)
    all3swioutnulldf.drop(9, inplace = True)
    all3swioutnulldf.reset_index(level = ['_INCOMG'])
    print (all3swioutnulldf.groupby(['_INCOMG']).count())'''
    
         
    
    all3swioutnulldf=seatwn
    #all3swioutnulldf.to_csv('seatbeltdata_modified.csv',)
    
    
    print ("Printing shape of seatbelt dataframe")
    #na_values = [' ', '.']
    s = seatdf.isnull().apply(sum, axis=0) #prints number of Number of NAN values in a column
    #print (s)
    print (seatdf.shape) #prints the number of columns and rows
    print ("After removing Na values in seatbelt prog")
    a = all3swioutnulldf.isnull().apply(sum, axis=0) #prints number of Number of NAN values in a column
    #print (a)
    print(all3swioutnulldf.shape) #prints the number of columns and rows'''


    print("Starting the analysis in Seatbelt Program")
    #all3swioutnulldf.reset_index(level = ['_AGEG5YR'])
    allhist = all3swioutnulldf
    
    groupcount = all3swioutnulldf['_RFSEAT3'].value_counts() #Group wise count
    print (groupcount)
    groupcount = all3swioutnulldf['_AGEG5YR'].value_counts() #Group wise count
    print (groupcount)
    groupcount = all3swioutnulldf['_EDUCAG'].value_counts() #Group wise count
    print (groupcount)
    groupcount = all3swioutnulldf['SEATBELT'].value_counts() #Group wise count
    print (groupcount)
    
    sns.set() #Following commands are to find the distribution using Histogram
    '''plt.hist(allhist['_EDUCAG'])
    plt.xlabel('Level of Education')
    plt.ylabel('People count')
    plt.show()'''
    #plt.savefig('Histogram')
    
    #sm.qqplot(all3swioutnulldf, loc = 4, scale = 3, line='s')
    print(stats.normaltest(allhist["_RFSEAT3"]))
    print(stats.normaltest(allhist["_AGEG5YR"]))
    
    agekru = allhist['_AGEG5YR']
    seatkru = allhist['_RFSEAT3']
    edukru = allhist['_EDUCAG']
    print("kruskal")
    h, pvalkru = stats.kruskal(seatkru,edukru)#Kruskal wallis test
    print (pvalkru)
    print (h)
    
    '''KRUSKAL TERMS
    if pval < 0.05:
    print("Reject NULL hypothesis - Significant differences exist between groups.")
    if pval > 0.05:
    print("Accept NULL hypothesis - No significant difference between groups.")
    if p > alpha:
    print('Same distributions (fail to reject H0)')
    else:
    print('Different distributions (reject H0)')'''
    
    
    
    print(stats.mannwhitneyu(all3swioutnulldf["_RFSEAT3"],all3swioutnulldf["_EDUCAG"]))#mannwhitney
    
    
        
    print (stats.f_oneway(all3swioutnulldf["_RFSEAT3"],all3swioutnulldf["_EDUCAG"])) # One way anova
    
           
    print ("Correlation between variables")
    limpy = all3swioutnulldf['_RFSEAT3'].corr(all3swioutnulldf['_EDUCAG'],method='pearson')# Corelation between variables. here Negative corelation
    print (limpy)
    
    ttestval = ttest_ind(all3swioutnulldf['_RFSEAT3'],all3swioutnulldf ['_EDUCAG']) #Ttest
    print (ttestval)
    
    print ("Linear regression model")
    x = all3swioutnulldf["_AGEG5YR"]
    y = all3swioutnulldf["_RFSEAT3"]
    linearregmodel = sm.OLS(y, x).fit()
    predictions = linearregmodel.predict(x) # make the predictions by the model
    
    # Print out the statistics
    print (linearregmodel.summary())
    
        
    '''df_out = all3swioutnulldf.groupby('_INCOMG').corr()
    tim = (df_out[~df_out['_AGEG5YR'].eq(1)].reset_index(1, drop=True)['_AGEG5YR'].rename('Corr_Coef').reset_index())
    print (tim)'''
    
    print ("Anova between variables")
    anval = ols('_RFSEAT3 ~ _EDUCAG', data=all3swioutnulldf).fit()#One way anova
    anova_table = sm.stats.anova_lm(anval, typ=2)
    print (anova_table)
    
    #moore_lm = ols('SEATBELT ~ C(_EDUCAG, Sum)*C(_AGEG5YR, Sum)', all3swioutnulldf).fit()
    model = ols('_RFSEAT3 ~ C(_AGEG5YR) + C(_EDUCAG)', all3swioutnulldf).fit()
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")
    res = sm.stats.anova_lm(model, typ= 2)
    print (res)
    
    print ("Printing Education details of group 3")
    
    
    
    ''''t = (all3swioutnulldf['_RFSEAT3']).count() #Count of total values in specific column
    print (t)
    groupcount = all3swioutnulldf['_RFSEAT3'].value_counts() #Group wise count
    print (groupcount)
    print("The percentile among groupwise is as follows,") # Percentage of the groups with the total value
    print (all3swioutnulldf['_AGEG5YR'].value_counts(normalize=True) * 100)'''
    
    
    '''ax = all3swioutnulldf.plot.bar(x=educol, y=dontwearseatcol, rot=0)#Bar plot for 2 variables
    plt.show()'''
    
    
    
    '''
    print("Printing box plot")
    boxdata = all3swioutnulldf.boxplot(column=['_EDUCAG','_AGEG5YR','SEATBELT'])#Boxplot All 3 values distribution
    plt.show()'''
    
    '''print ("GLM")
    model = sm.formula.glm("dontwearseatcol ~ C(_EDUCAG, Treatment(reference='3'))", family=sm.families.Binomial(), data=all3swioutnulldf).fit()
    print(model.summary())'''
            
    
    
'''
t = (all3swioutnulldf['_EDUCAG']).count() #Count of total values in specific column
print (t)

educol = all3wioutnulldf[['_EDUCAG']] #Save column details in a variable
    print (educol)

Matplot    
plt.plot(tamilarray, englisharray, label='Test')  
plt.show()

# filter out rows ina . dataframe with column year values NA/NAN
>gapminder_no_NA = gapminder[gapminder.year.notnull()]
edudetail = seatdf[seatdf._EDUCAG.notnull()] #To remove NA/NAan values from a column and form a dataframe

print (all3wioutnulldf._EDUCAG.dtype) To find the data type of the column

limpy = all3swioutnulldf['_AGEG5YR'].corr(all3swioutnulldf['_RFSEAT3'],method='pearson')

print (seatdf.head(2))
print(df['engmark']) #Prints the data in column
#tamilarray = edudetail.values

seatdf.dropna() #drop the Na values in whole data frame

#bxdata = all3wioutnulldf.boxplot(column=['_EDUCAG']) for box plot a data uisng pandas
bxdata = sns.boxplot(y='SEATBELT', x='_AGEG5YR', data=all3wioutnulldf, width=0.5, palette="colorblind")

#yespnevac = speciedu[speciedu == 9] passing condition

 ttestval = ttest_ind(educol, alwaysseatcol) #Ttest
 print (ttestval)
 
eng_col = len(df['engmark'])
print (eng_col) #prints total column number with NA

plt.scatter(all3swioutnulldf['_AGEG5YR'], all3swioutnulldf['SEATBELT']) #Scatter plot
    plt.show()
    
    ax = all3swioutnulldf.plot.bar(x='_EDUCAG', y='_RFSEAT3', rot=0)#Bar plot for 2 variables
    plt.show()
    
    sns.pairplot(all3swioutnulldf[["_EDUCAG", "_AGEG5YR", "SEATBELT"]])# Pairplot Compares all 3 variables
    plt.show()
   
df.to_csv('markdata_modified.csv',) #creates a file and add the content into it.
names = ['my_datetime', 'event', 'country', 'user_id', 'source', 'topic'] In order to name the column

graph = all3swioutnulldf[educol]
    test = graph[graph._RFSEAT3 == "2"]
    graph.Value.describe()
    test.plot()
    graph[graph._RFSEAT3 == "1"].plot()
    plt.show()
    
    
    y = all3swioutnulldf.loc[all3swioutnulldf['_RFSEAT3'] == 2, '_EDUCAG']
    print (y)'''
